In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

%run dots_model.ipynb
%run dots_ml_nnqplayer.ipynb
%run dots_training_data.ipynb
%run TFSessionManager.ipynb

def get_next_batch(x_set, y_set, start, end):
    return x_set[start : end], y_set[start : end]

def build_and_save_model(qplayer):
    print("building Q")
    qplayer = NNQPlayer("Q")
    nn = qplayer.nn

    epochs = 2            # Total number of training epochs
    batch_size = 10000        # Training batch size
    display_freq = 10000      # Frequency of displaying the training results

    # Create the op for initializing all variables
    init = tf.global_variables_initializer()

    accs = []
    iters = []

    starting_ind = 0

    print("starting session")
    # Launch the graph (session)
    #with tf.Session() as sess:
    sess = tf.Session()
    sess.run(init)
    writer = tf.summary.FileWriter('log', TFSessionManager.get_session().graph)
    global_step = 0
    # Number of training iterations in each epoch
    num_tr_iter = 100 #int(len(y_train) / batch_size)
    for epoch in range(epochs):
        print('Training epoch: {}'.format(epoch + 1))

        #x_train, y_train = randomize(x_train, y_train)
        for iteration in range(num_tr_iter):
            global_step += 1
            start = iteration * batch_size + starting_ind
            end = (iteration + 1) * batch_size + starting_ind
            #start = iteration * batch_size
            #end = (iteration + 1) * batch_size
            #x_batch, y_batch = get_next_batch(x_train, y_train, start, end)
            x_batch, y_batch = generateData(batch_size)
            # Run optimization op (backprop)
            feed_dict_batch = {nn.input_positions: x_batch, nn.target_input: y_batch}
            sess.run(nn.train_step, feed_dict=feed_dict_batch)

            if iteration % display_freq == 0:
                # Calculate and display the batch loss and accuracy
                acc_batch = sess.run([nn.accuracy], feed_dict=feed_dict_batch)

                accs.append(acc_batch)
                iters.append(global_step)

                print("iter {}:\t Training Accuracy={}".format(iteration, acc_batch))

    p = plt.plot(iters, accs, 'b-')
    plt.show()

    print('training data incorporated')
    
#build_and_save_model()
